In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext line_profiler

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import rydiqule as rq
from copy import deepcopy
import warnings


In [4]:
from rydiqule.sensor import Sensor
from rydiqule.sensor_utils import _hamiltonian_term, generate_eom, make_real, _squeeze_dims
from rydiqule.sensor_solution import Solution
from rydiqule.exceptions import RydiquleError, RydiquleWarning, PopulationNotConservedWarning
from rydiqule.doppler_exact import _doppler_eigvec_array
from importlib.metadata import version

In [5]:
def doppler_1d_exact_new(sensor: Sensor, rtol: float = 1e-5, atol: float = 1e-9) -> Solution:
    """
    Analytically solves a sensor in steady-state in the presence of 1 dimensional
    Doppler broadening.

    Uses the method outlined in Ref [1]_.
    In particular, it uses Eq. 12 to analytically evaluate the Doppler average in 1D.

    This solver is considered more accurate than :func:`~.solve_steady_state`
    since it replaces direct sampling and solving of the velocity classes
    with a few tensor inversions and calculation of the numerical prefactor.
    This also leads to faster solves,
    approximately dictated by the ratio of samples along the doppler axis
    relative to the other parameter dimensions.

    Parameters
    ----------
    sensor : :class:`~.Sensor`
        The sensor for which the solution will be calculated.
        It must define 1 and only 1 dimension of doppler shifts
        (ie one or more couplings with `kvec` with non-zero values on the same dimension).
    rtol: float, optional
        Relative tolerance parameter for checking 0-eigenvalues when calculating the doppler prefactor.
        Passed to :external+numpy:func:`~numpy.isclose`.
        Defaults to 1e-5.
    atol: float, optional
        Absolute tolerance parameter for checking 0-eigenvalues when calculating the doppler prefactor.
        Passed to :external+numpy:func:`~numpy.isclose`.
        Defaults to 1e-9.

    Returns
    -------
    :class:`~.Solution`
        An object containing the solution and related information.

    Raises
    ------
    RydiquleError
        If the `sensor` does not have exactly 1 dimension of doppler shifts to average over.
    AssertionError
        If the initial rho0 calculation results in an unphysical result.

    Warns
    -----
    RydiquleWarning
        If the averaged result is not real within tolerances.
        While rydiqule's computational basis is real,
        the method employed here involves complex number floating point calculations.
        If all is well, the complex parts should cancel to return a real result,
        but imprecision in floating point operations can occur.
    PopulationNotConservedWarning
        Before removing the ground state in the final solution,
        population conservation is confirmed.
        If the resulting density matrices do not preserve trace, this warning is raised
        indicating an issue in the calculation.

    References
    ----------
    .. [1] Omar Nagib and Thad G. Walker,
        Exact steady state of perturbed open quantum systems,
        arXiv 2501.06134 (2025)
        http://arxiv.org/abs/2501.06134
    """

    if sensor.spatial_dim() != 1:
        raise RydiquleError(f'Sensor must have 1 spatial dimension of Doppler shifts, found {sensor.spatial_dim():d}')

    stack_shape = sensor._stack_shape()
    n = sensor.basis_size
    # Liouvillian superoperator for the non-doppler-broadened components
    L0, dummy_const = generate_eom(sensor.get_hamiltonian(), sensor.decoherence_matrix(),
                      remove_ground_state=False,
                      real_eom=True)

    
    ### Calculate rho0 via inverse power method and construct L0^minus using new Nagib/Walker method
    rho0 = np.random.rand(stack_shape[0]*stack_shape[1], n**2)[..., np.newaxis]
    rho0 /= np.linalg.norm(rho0, axis=1, keepdims=True)

    I = np.eye(n**2)
    converged_flags = np.zeros(stack_shape[0]*stack_shape[1], dtype=bool)
    L0_flat = L0.reshape(stack_shape[0]*stack_shape[1], n**2, n**2)
        
    # Compute rho0 by finding the null vector of L0 via the shifted inverse power method
    for iteration in range(50):
        if np.all(converged_flags):
            break
        
        remaining_flags_index = np.where(~converged_flags)[0]
        current_L0 = L0_flat[remaining_flags_index]
        current_shifted_L0 = current_L0 + 1e-14 * I
    
        current_rho0 = rho0[remaining_flags_index]
         
        z = np.linalg.solve(current_shifted_L0, current_rho0) # compute (L0 + 1e-14)^-1 * rho0
        rho0_new = z / (np.linalg.norm(z, axis=1, keepdims=True) + np.finfo(float).eps)
        
        # Estimate magnitude of eigenvalues by Rayleigh quotient
        L0rho0 = np.einsum('nij,nj->ni', current_L0, rho0_new[..., 0])
        numerator = np.sum( rho0_new[..., 0] * L0rho0, axis=1)
        denominator = np.sum( rho0_new[..., 0] * rho0_new[..., 0], axis=1)
        current_eigenvalues = np.abs(numerator / denominator)
    
        rho_0_diff = np.linalg.norm(current_rho0 - rho0_new, axis = 1)
    
        converged_flags[remaining_flags_index] = (rho_0_diff.flatten() < 1e-15) | (current_eigenvalues < 1e-14)
              
        rho0[remaining_flags_index] = rho0_new

    rho0 = rho0.reshape(stack_shape[0], stack_shape[1], n**2)

    assert not np.iscomplexobj(rho0), 'rho0 solution is not real; it is unphysical'
    rho0 *= np.sign(rho0[...,0])[...,None]  # remove arbitrary sign from null-vector so all pops are positive
    pops = np.sum(rho0[...,::n+1], axis=-1)  # calculate trace of each vector
    rho0 /= pops[..., None]  # normalize vectors by trace

    vec1 = np.eye(n).flatten() #Initialize vectorized identity
    L0m = np.linalg.inv(L0 + rho0[..., :, np.newaxis] * vec1[np.newaxis, np.newaxis, :]) - rho0[..., :, np.newaxis] * vec1[np.newaxis, np.newaxis, :]
    
    ### Liouvillian superoperator for doppler only
    # these are already multiplied by sqrt(2)*sigma_v by rydiqule
    # as such, lambdas are redefined as sqrt(2)*sigma_v*lambdas of Eq12 in the paper
    dopp = sensor.get_doppler_shifts().squeeze()
    Lv_complex = _hamiltonian_term(dopp)
    Lv, _ = make_real(Lv_complex, dummy_const, ground_removed=False)

    ### Calculate doppler averaged steady-state density matrix from propagator
    el, R = np.linalg.eig(L0m@Lv)
    L = np.linalg.pinv(np.swapaxes(R,-1,-2))

    # calculate Eq 12
    prefix = _doppler_eigvec_array(el, rtol, atol)
    rho_dopp_complex = np.einsum('...j,...ij,...kj,...k->...i', prefix, R, L, rho0)
    # confirm that result is approximately real
    imag_tol = 10000
    rho_dopp = np.real_if_close(rho_dopp_complex, tol=imag_tol)  # chop complex parts if all are smaller than 10000*f64_eps
    if np.iscomplexobj(rho_dopp):
        rho_dopp_imag = np.abs(rho_dopp_complex.imag)
        count = np.count_nonzero(rho_dopp_imag > np.finfo(float).eps*imag_tol)
        warnings.warn('Doppler-averaged solution has complex parts outside of tolerance, solution is suspect. ' +
                      f'{count:d} of {rho_dopp.size:d} elments larger than cutoff {np.finfo(float).eps*imag_tol:.3g}. ' +
                      f'Max Abs(Imag): {rho_dopp_imag.max():.3g}, Std Abs(Imag): {np.std(rho_dopp_imag):.3g}',
                      RydiquleWarning)
    # ensure trace is preserved before dropping ground state, which will implicitely enforce it
    pops_dopp = np.sum(rho_dopp[...,::n+1], axis=-1)
    if not np.isclose(pops_dopp, 1.0).all():
        warnings.warn('Doppler-averaged solution has populations not conserved, solution is suspect. ' +
                      f'{np.count_nonzero(~np.isclose(pops_dopp, 1.0)):d} of {pops_dopp.size:d} have non-unit trace. ' +
                      f'Min trace is {pops_dopp.min():f}',
                      PopulationNotConservedWarning)

    ### make into a Solution object
    solution = Solution()
    # match rydiqule convention for return (ie no ground population)
    solution.rho = rho_dopp[...,1:]
    
    # specific to observable calculations
    solution._eta = sensor.eta
    solution._kappa = sensor.kappa
    solution._cell_length = sensor.cell_length
    solution._beam_area = sensor.beam_area
    solution._probe_freq = sensor.probe_freq
    solution._probe_tuple = sensor.probe_tuple

    # store the graph with fully_expanded dimensionality
    sensor._expand_dims()
    solution.couplings = deepcopy(sensor.couplings)
    _squeeze_dims(sensor.couplings)

    solution.axis_labels = (sensor.axis_labels() + ["density_matrix"])
    solution.axis_values = ([val for _,_,val,_ in sensor.variable_parameters()]
                            + [sensor.dm_basis()])
    solution.dm_basis = sensor.dm_basis()
    solution.rq_version = version("rydiqule")

    return solution

In [6]:
# parameters for Cell
atom = 'Rb87'

states = [
    rq.ground_state(atom),
    rq.D2_excited(atom),
    rq.A_QState(41,2,5/2),
    rq.A_QState(40,3,7/2)
]


cell = rq.Cell(atom, states)

In [7]:
# laser parameters
detunings = 2*np.pi*np.linspace(-30,30,201)
Omega_r = 2*np.pi*2
Omega_b = 2*np.pi*5
Omega_rf = 2*np.pi*np.array([0,5,40])

kunit1 = np.array([1,0,0])
kunit2 = np.array([-1,0,0])

red = {'states': (states[0],states[1]), 'detuning': detunings, 'rabi_frequency': Omega_r, 'kunit': kunit1}
blue = {'states': (states[1],states[2]), 'detuning': 0, 'rabi_frequency': Omega_b, 'kunit': kunit2}
rf = {'states': (states[2],states[3]), 'detuning': 0, 'rabi_frequency': Omega_rf}

cell.add_couplings(red, blue, rf)

In [14]:
%%time
sol_old = rq.doppler_1d_exact(cell)

CPU times: total: 953 ms
Wall time: 194 ms


In [13]:
%%time
sol_new = doppler_1d_exact_new(cell)

CPU times: total: 109 ms
Wall time: 131 ms


In [16]:
%timeit sol_old = rq.doppler_1d_exact(cell)

216 ms ± 15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%timeit sol_new = doppler_1d_exact_new(cell)

96.6 ms ± 1.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
%lprun -f rq.doppler_1d_exact rq.doppler_1d_exact(cell)

Timer unit: 1e-07 s

Total time: 0.243231 s
File: c:\Users\RydbergControl\miniconda3\envs\rydiqule2\Lib\site-packages\rydiqule\doppler_exact.py
Function: doppler_1d_exact at line 69

Line #      Hits         Time  Per Hit   % Time  Line Contents
    69                                           def doppler_1d_exact(sensor: Sensor, rtol: float = 1e-5, atol: float = 1e-9) -> Solution:
    70                                               """
    71                                               Analytically solves a sensor in steady-state in the presence of 1 dimensional
    72                                               Doppler broadening.
    73                                           
    74                                               Uses the method outlined in Ref [1]_.
    75                                               In particular, it uses Eq. 12 to analytically evaluate the Doppler average in 1D.
    76                                           
    77                      

In [12]:
from doppler_exact_new import doppler_1d_exact_new

%lprun -f doppler_1d_exact_new doppler_1d_exact_new(cell)

Timer unit: 1e-07 s

Total time: 0.109478 s
File: c:\Users\RydbergControl\Documents\GitHub\Rydiqule\src\rydiqule\doppler_exact_new.py
Function: doppler_1d_exact_new at line 70

Line #      Hits         Time  Per Hit   % Time  Line Contents
    70                                           def doppler_1d_exact_new(sensor: Sensor, rtol: float = 1e-5, atol: float = 1e-9) -> Solution:
    71                                               """
    72                                               Analytically solves a sensor in steady-state in the presence of 1 dimensional
    73                                               Doppler broadening.
    74                                           
    75                                               Uses the method outlined in Ref [1]_.
    76                                               In particular, it uses Eq. 12 to analytically evaluate the Doppler average in 1D.
    77                                           
    78                        

In [13]:
rq.about()


        Rydiqule
    
Rydiqule Version:     2.0.0
Installation Path:    ~\miniconda3\envs\rydiqule2\Lib\site-packages\rydiqule

      Dependencies
    
NumPy Version:        2.0.1
SciPy Version:        1.15.3
Matplotlib Version:   3.10.0
ARC Version:          3.8.1
Python Version:       3.12.11
Python Install Path:  ~\miniconda3\envs\rydiqule2
Platform Info:        Windows (AMD64)
CPU Count and Freq:   16 @ 3.91 GHz
Total System Memory:  256 GB
